<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Деление-на-выборки-и-векторизация-слов" data-toc-modified-id="Деление-на-выборки-и-векторизация-слов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Деление на выборки и векторизация слов</a></span></li><li><span><a href="#Подбор-гиперпараметров-SVM" data-toc-modified-id="Подбор-гиперпараметров-SVM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Подбор гиперпараметров SVM</a></span></li><li><span><a href="#Подбор-гиперпараметров-LogReg" data-toc-modified-id="Подбор-гиперпараметров-LogReg-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Подбор гиперпараметров LogReg</a></span></li><li><span><a href="#SVM-на-тесте" data-toc-modified-id="SVM-на-тесте-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>SVM на тесте</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [15]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt
import tqdm
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data = pd.read_csv("/datasets/toxic_comments.csv", index_col=0 )

In [4]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


In [5]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

data['text'] = data['text'].map(cleaning)

In [7]:
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
3,more i cant make any real suggestions on impro...,0
4,you sir are my hero any chance you remember wh...,0


In [11]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    lemm_text = " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    return lemm_text

In [12]:
tqdm.pandas()
data['lemm_text'] = data['text'].progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

**Вывод:** В ходе подготовки датасета твиты были очищены и лемматизированы. При лемматизации я использовал tqdm для того чтобы отслеживать процесс в реальном времени.

## Обучение

### Деление на выборки и векторизация слов

In [13]:
features=data['text']
target=data['toxic']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size=0.25, 
                                                    random_state=42, 
                                                    stratify = target)

In [210]:
stopwords = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words = stopwords)
X_train = count_tf_idf.fit_transform(X_train)
X_test = count_tf_idf.transform(X_test)

### Подбор гиперпараметров SVM

In [17]:
svc = LinearSVC(random_state=0, tol=1e-5)

In [18]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", svc),
    ]
)

In [25]:
parameter_grid = {
    "clf__C": list(range(1,15,3))
}

In [29]:
grid = GridSearchCV(
    pipeline, 
    parameter_grid,  
    cv=3, 
    refit = True, 
    verbose=1, 
    scoring = 'f1')

In [30]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        LinearSVC(random_state=0, tol=1e-05))]),
             param_grid={'clf__C': [1, 4, 7, 10, 13]}, scoring='f1', verbose=1)

In [31]:
grid.best_score_

0.7823631316984551

In [32]:
grid.best_params_

{'clf__C': 1}

### Подбор гиперпараметров LogReg

In [35]:
lr= LogisticRegression(solver='liblinear', max_iter=200, class_weight='balanced')

In [36]:
param_grid ={
    'clf__penalty':['l1','l2'],
    'clf__C':list(range(1,15,3))
}

In [37]:
pipeline_lr = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", lr),
    ]
)

In [38]:
grid_search_lr = GridSearchCV(pipeline_lr,
                          param_grid,
                          n_jobs=-1,
                          cv=3,
                          scoring='f1',
                          verbose=1)

In [40]:
grid_search_lr.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=200,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'clf__C': [1, 4, 7, 10, 13],
                         'clf__penalty': ['l1', 'l2']},
             scoring='f1', verbose=1)

In [42]:
grid_search_lr.best_score_

0.7767874058022334

In [44]:
#grid_search_lr.best_params_

{'clf__C': 4, 'clf__penalty': 'l1'}

### SVM на тесте

In [46]:
pipeline_final = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", LinearSVC(random_state=0, tol=1e-5, C=1)),
    ]
)

In [47]:
pipeline_final.fit(X_train, y_train)
pred = pipeline_final.predict(X_test)
f1_score(y_test, pred)

0.7860105059441528

## Выводы

В ходе данного проекта я анализировал отзывы на токсичность. Для этого использовалась tf_idf модель представления языка.
1. Сначала я очистил и лемматизировал изначальный текст.
2. Затем я поделил датасет на трейн и тес, используя аргумент stratify для сохранения распредления токсичных отзывов.
2. Далее я изучил гиперпараметры для двух моделей.

LinearSVC дал лучшие результаты на кросс-валидации. На тесте LinearSVC значение **f1 = 0.786**, что является приемлимым результатом в рамках этого проекта